# Lab 7 clasificador KNN

In [1]:
import pandas as pd

In [2]:
parkinsons = pd.read_csv("dataset/parkinsons_disease_data.csv")
diabetes = pd.read_csv("dataset/diabetes.csv")

In [6]:
featuresDiabetes = diabetes
featuresParkinsons = parkinsons[['UPDRS','FunctionalAssessment','MoCA','Tremor','Rigidity','Bradykinesia','Age','DietQuality','CholesterolTotal','CholesterolLDL','CholesterolTriglycerides','SystolicBP','CholesterolHDL','AlcoholConsumption','Diagnosis']]

,UPDRS,FunctionalAssessment,MoCA,Tremor,Rigidity,Bradykinesia,Age,DietQuality,CholesterolTotal,CholesterolLDL,CholesterolTriglycerides,SystolicBP,CholesterolHDL,AlcoholConsumption,Diagnosis
0,6.458713,1.572427,29.181289,1,0,0,85,3.893969,222.842278,148.125623,337.307114,129,37.867776,5.108241,0
1,37.306703,4.787551,12.332639,0,1,0,75,8.513428,210.501133,153.756463,264.635521,163,77.228117,6.027648,1
2,67.838170,2.130686,29.927783,1,0,0,70,6.498805,287.387996,118.702600,395.662649,113,85.588304,2.242135,1
3,52.964696,3.391288,21.304268,1,1,1,52,6.715033,280.339546,136.299186,362.189688,146,51.869626,5.997788,1
4,21.804880,3.200969,8.336364,0,0,0,87,4.657572,284.014151,108.449452,149.956586,115,25.069418,9.775243,0
